10h, 1/h 5/h 9/h 几种档位, 如何最省的情况下, 达到 33 的水

$$
min \sum_{h=0}^\infty cost_h^m
$$

s.t.
$$ \sum_{h=0}^\infty flow_h^m >= 33 $$

In [20]:
import pandas as pd
from docplex.mp.model import Model

hourArr = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  # 10 个小时
modeArr = [0, 1, 5, 9] # 4 档位，【关，小，中，大】
mCost = {
    0: 0,
    1: 2,
    5: 6,
    9: 9,
}

model = Model(name="Demo")

# x0 = model.integer_var_dict(hourArr, lb=0, ub=2, name="h")
x0 = {(h, m): model.binary_var(name=f"h:{h}-m:{m}") for h in hourArr for m in modeArr}

# cons
model.add_constraint(
    ct=model.sum(x0[one] for one in x0) <= 10,
    ctname="cons_0",
)
model.add_constraint(
    ct=model.sum(x0[h, m] * m for h, m in x0) >= 33,
    ctname=f"cons_100",
)
for h in hourArr:
    model.add_constraint(
        ct=model.sum(x0[h, m] for m in modeArr) == 1,
        ctname=f"cons_h_{h}",
    )

# target
model.minimize(model.sum(x0[h, m] * mCost[m] for h, m in x0))
model.solve() # log_output=True
model.print_solution()

objective: 35
status: OPTIMAL_SOLUTION(2)
  h:0-m:0=1
  h:1-m:1=1
  h:2-m:0=1
  h:3-m:0=1
  h:4-m:0=1
  h:5-m:0=1
  h:6-m:5=1
  h:7-m:9=1
  h:8-m:9=1
  h:9-m:9=1


In [2]:
model.print_information()

# output
df = pd.DataFrame.from_dict(x0, orient="index", columns=["variable_object"])
df.index = pd.MultiIndex.from_tuples(df.index, names=["hour", "flow"])
df.reset_index(inplace=True)

df["solution_value"] = df["variable_object"].apply(lambda item: item.solution_value)
df.drop(columns=["variable_object"], inplace=True)
# print(df)